In [1]:
import numpy
from scipy.fftpack import dct
from scipy.io import wavfile
import pandas as pd
import numpy as np
from python_speech_features import mfcc
import os
import re
from numpy import newaxis
from script import *
fol_input= 'data'
#rootdir='enter_directory/bengali/'
fol_output ='output'


c:\anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
data,y=extract_features(rootdir,fol_output)           


In [4]:
data[1:10]


,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,68
1,0.375,220.0,4630.3955,0.375,21.351954,-0.719815,-8.196278,26.936718,-19.449858,-17.036129,...,-0.212878,-1.243805,-1.012372,-1.102883,0.859878,-0.084953,0.164384,-0.182059,0.441208,-0.898221
2,0.375,233.0,2461.6045,0.375,20.098391,1.551649,1.592873,16.977976,-19.883131,0.615746,...,0.246557,-1.505821,-0.049629,-0.571831,0.283111,0.730313,-0.113071,0.000293,-0.156661,-0.418779
3,0.250,149.0,3584.3870,0.250,20.956120,-2.268425,1.146147,14.091915,-29.660058,5.689657,...,0.132114,-2.483086,0.116573,-0.980473,0.130590,1.044733,0.541760,-0.063003,-0.086317,-0.478497
4,0.250,137.0,2808.2725,0.250,20.332793,-8.685330,-1.751928,2.360482,-21.104983,-15.582627,...,-0.713040,-2.114468,-1.112978,-0.756730,1.572786,-0.702936,1.092423,-0.651345,0.369725,-1.030937
5,0.375,218.0,5847.5745,0.375,21.062612,4.713172,-4.133764,21.734452,-21.487389,-14.077373,...,-0.085253,-0.646281,-0.677582,-0.464765,0.664453,-0.224184,0.943724,-0.437191,0.420615,-0.517588
6,0.625,97.0,3088.1430,0.625,21.830339,9.826351,10.550861,17.904464,17.368982,13.733715,...,-0.166994,-0.160295,-0.086541,0.075274,-0.028367,-0.117457,-0.220260,-0.140531,0.052672,-0.150371
7,0.375,272.0,4914.3295,0.375,22.482414,-3.667647,10.320054,16.501035,0.145729,-12.272070,...,0.052838,-0.136798,-0.346180,0.002503,-0.205358,-0.020771,0.035203,-0.078086,-0.342679,0.298173
8,0.375,161.0,7095.3445,0.375,21.256248,5.459345,19.091223,34.818831,8.351679,-3.461627,...,0.492791,-0.659160,-1.078929,-0.096979,-0.172090,0.034703,-0.630295,0.539300,-0.412699,-0.075829
9,0.375,112.0,5718.6200,0.375,22.381089,5.495798,13.439258,35.754372,9.829241,-2.309807,...,0.481407,0.194590,-0.331913,0.099226,-0.589532,-0.573803,0.113955,-0.704119,-0.609228,-0.230985


In [9]:

data.shape


array([1, 2])

In [10]:
#from numpy import newaxis
X_train=np.array(data[0:8000])
X_train=X_train[...,newaxis]

y_train=np.array(y[0:8000])

X_test=np.array(data[8000:10000])

X_test=X_test[...,newaxis]
y_test=np.array(y[8000:10000])


In [12]:
print np.bincount(y_test)
print np.bincount(y_train)

[194 204 195 172 196 230 163 203 201 242]
[774 837 783 762 788 846 732 796 803 879]


In [13]:
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.layers import Dropout
np.random.seed(1337)

Using Theano backend.


In [14]:
batch_size = 5
hidden_units = 100
nb_classes = 10

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [15]:
print X_train.shape
print Y_train.shape

print X_train.shape[1:]
print nb_classes

(8000L, 69L, 1L)
(8000L, 10L)
(69L, 1L)
10


In [16]:
model = Sequential()
model.add(LSTM(output_dim=hidden_units, init='uniform', inner_init='uniform',
               forget_bias_init='one', 
               activation='tanh', 
               inner_activation='sigmoid', 
               input_shape=X_train.shape[1:]))
model.add(Dropout(0.1))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [17]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=30, validation_data=(X_test, Y_test), show_accuracy=True)


Train on 8000 samples, validate on 2000 samples
Epoch 1/30
8000/8000 [==============================] - 84s - loss: 2.2402 - acc: 0.1320 - val_loss: 2.1215 - val_acc: 0.1980
Epoch 2/30
8000/8000 [==============================] - 91s - loss: 1.9644 - acc: 0.2138 - val_loss: 1.9685 - val_acc: 0.2260
Epoch 3/30
8000/8000 [==============================] - 92s - loss: 1.8451 - acc: 0.2724 - val_loss: 1.8633 - val_acc: 0.2920
Epoch 4/30
8000/8000 [==============================] - 75s - loss: 1.6793 - acc: 0.3720 - val_loss: 1.6886 - val_acc: 0.3700
Epoch 5/30
8000/8000 [==============================] - 89s - loss: 1.5138 - acc: 0.4476 - val_loss: 1.5742 - val_acc: 0.4460
Epoch 6/30
8000/8000 [==============================] - 75s - loss: 1.4049 - acc: 0.4963 - val_loss: 1.3251 - val_acc: 0.5470
Epoch 7/30
8000/8000 [==============================] - 78s - loss: 1.3055 - acc: 0.5309 - val_loss: 1.2020 - val_acc: 0.5755
Epoch 8/30
8000/8000 [==============================] - 76s - loss: 1.

c:\anaconda2\lib\site-packages\keras\models.py:610: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


In [18]:
score, acc = model.evaluate(X_test.astype(numpy.float32, copy=False), Y_test.astype(numpy.float32, copy=False),
                            batch_size=batch_size,
                            show_accuracy=True)
print acc

2000/2000 [==============================] - 7s     
0.917000004165


c:\anaconda2\lib\site-packages\keras\models.py:651: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


In [36]:
#Y_pred=model.predict(X_test)